<a href="https://colab.research.google.com/github/aruaru0/pytorch-tests/blob/main/search_wikipedia_langchain_CALM2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 95.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install langchain sentence_transformers
!pip install faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Attempting uninstall: toke

In [5]:
# !CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

# LLMの読み込み（CALM2を使ったが他のモデルでもhuggingfaceならOKだと思う）

In [6]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer


model = AutoModelForCausalLM.from_pretrained("cyberagent/calm2-7b-chat",
                                             device_map="auto",
                                             torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained("cyberagent/calm2-7b-chat")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

model.eval()

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.04G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(65024, 4096, padding_idx=1)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
 

# wikipediaからデータを取得する

In [7]:
import requests

def get_wikipedia_page(keyword):
  url = "https://ja.wikipedia.org/w/api.php"
  params = {
      "action": "query",
      "list":"search",
      "srsearch": keyword,
      "srlimit":1,
      # "srnamespace": 1,
      "format": "json",
  }

  response = requests.get(url, params=params)
  data = response.json()
  id = data['query']['search'][0]['pageid']

  params = {
    "action": "query",
    "format": "json",
    "pageids": id,
    "prop": "extracts",
    "exlimit": 1,
    "explaintext": True,
    "redirects": 1,
  }

  response = requests.get(url, params=params)
  data = response.json()
  page = next(iter(data['query']['pages'].values()))

  return page.get("extract", "")

In [49]:
wiki_txt = get_wikipedia_page("ワンパンマン")
len(wiki_txt), wiki_txt[:1000]

(115958,
 '『ワンパンマン』は、ONEによる日本の漫画作品。2009年7月3日からウェブサイト上で連載され、2012年9月2日放送のNHK『ネット発 マンガ革命』によれば、1日2万回閲覧され累計1000万人以上が閲覧したという。\nリメイク版（作画：村田雄介）が『となりのヤングジャンプ』（集英社）において2012年6月14日から連載されている。リメイク版で併記されている英語タイトルは『One Punch-Man』。\n\n\n== 概要 ==\nあらゆる敵をパンチ1発（ワンパン）で倒してしまう、最強ヒーローサイタマが主人公のギャグ系アクション漫画。話数カウントは「○撃目」。\n2011年6月24日までウェブコミック投稿サイト「新都社」に登録されていた。現在も作品ページ（ONEのホームページ）は存続しており更新継続中である。\n本作は当初、ONEがパソコン用の漫画原稿制作ソフトComicStudioの練習のために執筆したものであった。しかし、読者からの反応が予想より大きかったことを受けて最終話までのプロットを制作し、現在はそれに従い制作しているという。\n村田雄介の作画によるリメイク版が『となりのヤングジャンプ』（集英社）において2012年6月14日から連載されている。\n2015年3月12日発売の『週刊ヤングジャンプ』15号にて、リメイク版のテレビアニメ化が発表された。2016年3月には、第2回SUGOI JAPAN Awardでマンガ部門1位を獲得した。\n原作は2017年1月から更新が止まっていたが、2017年9月に作者のONEが原作（ONE版）の更新休止をホームページ上にて正式に発表。その後2019年4月8日に更新を再開した。\n2020年、アメリカのヴァラエティ誌は『ワンパンマン』がソニー・ピクチャーズ制作の下ハリウッドで実写映画化される予定があると報じた。同誌によると脚本を担当するのは『ヴェノム』のスコット・ローゼンバーグとジェフ・ピンクナーでありプロデューサーはアヴィ・アラッドであるという｡\n単行本の累計発行部数は2024年3月時点で3000万部を突破している。\n以前よりキャラクターデザインの変更など描き直しが多い作品であるが、近年（2020年時点）は単行本化前にストーリー自体を変更させる描き直しが増えており、当初描かれた話では死んでいたキ

#　Faissのデータベースを作成

In [50]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.faiss import FAISS

# CALM2は32000トークンまで対応しているので、5000文字と少し大きめのサイズで区切っている
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=500,
)
texts = text_splitter.split_text(wiki_txt)

In [51]:
embedding=HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

index = FAISS.from_texts(
    texts=texts,
    embedding=embedding
)
index.save_local("storage")

In [52]:
txt = "無限パンチとはなんですか？"
docs = index.similarity_search(txt)
docs[0]

Document(page_content='『ワンパンマン』は、ONEによる日本の漫画作品。2009年7月3日からウェブサイト上で連載され、2012年9月2日放送のNHK『ネット発 マンガ革命』によれば、1日2万回閲覧され累計1000万人以上が閲覧したという。\nリメイク版（作画：村田雄介）が『となりのヤングジャンプ』（集英社）において2012年6月14日から連載されている。リメイク版で併記されている英語タイトルは『One Punch-Man』。\n\n\n== 概要 ==\nあらゆる敵をパンチ1発（ワンパン）で倒してしまう、最強ヒーローサイタマが主人公のギャグ系アクション漫画。話数カウントは「○撃目」。\n2011年6月24日までウェブコミック投稿サイト「新都社」に登録されていた。現在も作品ページ（ONEのホームページ）は存続しており更新継続中である。\n本作は当初、ONEがパソコン用の漫画原稿制作ソフトComicStudioの練習のために執筆したものであった。しかし、読者からの反応が予想より大きかったことを受けて最終話までのプロットを制作し、現在はそれに従い制作しているという。\n村田雄介の作画によるリメイク版が『となりのヤングジャンプ』（集英社）において2012年6月14日から連載されている。\n2015年3月12日発売の『週刊ヤングジャンプ』15号にて、リメイク版のテレビアニメ化が発表された。2016年3月には、第2回SUGOI JAPAN Awardでマンガ部門1位を獲得した。\n原作は2017年1月から更新が止まっていたが、2017年9月に作者のONEが原作（ONE版）の更新休止をホームページ上にて正式に発表。その後2019年4月8日に更新を再開した。\n2020年、アメリカのヴァラエティ誌は『ワンパンマン』がソニー・ピクチャーズ制作の下ハリウッドで実写映画化される予定があると報じた。同誌によると脚本を担当するのは『ヴェノム』のスコット・ローゼンバーグとジェフ・ピンクナーでありプロデューサーはアヴィ・アラッドであるという｡\n単行本の累計発行部数は2024年3月時点で3000万部を突破している。\n以前よりキャラクターデザインの変更など描き直しが多い作品であるが、近年（2020年時点）は単行本化前にストーリー自体を変更させる描き直しが増えており、当初描

## 自分でRAGをする例

In [53]:
txt = "アクエリオンのあらすじを教えてください"

docs = index.similarity_search(txt)[0]

prompt = f"""USER: あなたは以下の情報に詳しいアシスタントです。わかりやすい日本語で答えてください。
以下の情報が質問に関連していない場合は、情報を利用せずに質問に答えてください。

情報：
{docs.page_content}

質問: {txt}
ASSISTANT:"""

input_ids = tokenizer.encode(prompt, return_tensors="pt")

print(f"\n文字数={len(prompt)}, トークン数={len(input_ids[0])}")

print("アシスタント：", end=" ")
with torch.no_grad():
  tokens = model.generate(
    input_ids = input_ids.to(model.device),
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.7,
    top_k = 50,
    streamer=streamer,
  )


文字数=5118, トークン数=2814
アシスタント：  アクエリオンは、全26話の日本のアニメ作品です。2005年4月から6月まで毎日放送と、同年10月から12月まで独立UHF局（テレビ神奈川、テレビ埼玉、千葉テレビ、三重テレビ、岐阜放送、サンテレビ、KBS京都）で放映されました。

物語は、地球と異星人「ディーバ」との戦争から始まります。主人公の少年・ショウは、異星人が地球に送り込んだ「アクエリア」というロボットのパイロットに選ばれ、ディーバとの戦争に身を投じます。

アクエリアは、異星人の技術で作られた巨大ロボットで、エネルギーとして「シャインスパーク」という特殊能力を使うことができます。ショウはアクエリアのパイロットとして、ディーバとの戦闘で活躍しますが、その中で「シャインスパーク」の秘密を知る人物、ユノと交流を持ちます。

ユノは、ディーバの王女であり、彼女が持つ特殊な力「エレメント能力」により、ディーバと地球との共存を模索していました。しかし、ディーバが地球に送り込んだ侵略用ロボット「アクエリオン」の侵攻により、ユノはディーバの王位継承権を放棄し、地球を守る戦いを決意します。

アクエリオンは、ディーバが開発した地球侵略用ロボットであり、強力なエネルギー「ロゴス」を持つ人間を求めます。ディーバはその人間を選び出すため、「エレメント能力」を持つ人間を地球に送り込み、ショウと接触させます。

ショウは、ユノと出会い、アクエリオンのパイロットとして共に戦うことを決意します。アクエリオンは、ロゴスを持つ人間と合体することが可能で、ショウはユノと2機目のアクエリオン「アクエリオンスパーダ」で戦います。

アクエリオンスパーダは、ロゴスにより飛行能力を得ることができ、地上、空中、宇宙空間と、あらゆる場所で戦うことが可能となります。また、アクエリオンスパーダには、武器として「アルテア剣」という特殊剣が搭載され、ロゴスを吸収して攻撃力を高めることができます。

最終的に、アクエリオンスパーダはロゴスを使い果たし、ディーバの女王・ミコノと融合して、ディーバの女王として地球を守ることを宣言します。


# Langchainを使ってstuffしてみる

In [54]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline


pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1000,
    do_sample=True,
    temperature=0.7,
    top_k = 50,
)
local_llm = HuggingFacePipeline(pipeline=pipe)

In [55]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain import PromptTemplate

# qa_chain = (
#     {'question': RunnablePassthrough()}
#     | PromptTemplate.from_template(base_template)
#     | local_llm
#     | StrOutputParser()
# )

# print(res)

In [56]:
base_template = """USER: あなたは以下の情報に詳しいアシスタントです。わかりやすい日本語で答えてください。
以下の情報が質問に関連していない場合は、情報を利用せずに質問に答えてください。

情報：
{context}

質問: {question}
ASSISTANT:"""

prompt = PromptTemplate.from_template(base_template)

qa_chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    retriever=index.as_retriever(search_kwargs={"k": 2}),
    verbose=True,
)
res = qa_chain.invoke('アクエリオンのアニメシリーズはいくつありますか？')
print(res['result'])



> Entering new RetrievalQA chain...

> Finished chain.
 アクエリオンのアニメシリーズは「アクエリオンEVOL」と「アクエリオン1980」の2つです。


In [62]:
res = qa_chain.invoke('ワンパンマンの敵のランクを教えてください')
print(res['result'])



> Entering new RetrievalQA chain...

> Finished chain.
 ワンパンマンに登場する怪人のランクは、災害レベルによって表現されます。

災害レベルは、人間が脅威と感じる程度によってランク付けされており、数字が大きくなるほど大きな被害が発生します。

以下に、ワンパンマンの敵のランクを、災害レベルによって示します。

* 災害レベル「竜」：怪人の中でも特に強い存在で、人間であればほとんど死亡する。
* 災害レベル「鬼」：怪人によって人間が大量殺害されるレベル。
* 災害レベル「虎」：怪人によって人間が負傷し、住処を追われるレベル。
* 災害レベル「竜」程度の強さを持つ怪人：竜に匹敵する脅威を持つが、まだ対処可能なレベル。
* 災害レベル「鬼」程度の強さを持つ怪人：鬼に匹敵する脅威を持つが、対処可能な範囲内であるレベル。
* 災害レベル「虎」程度の強さを持つ怪人：虎に匹敵する脅威を持つが、対処可能な範囲内であるレベル。
* 災害レベル「竜」未満の怪人：脅威度は低いが、対処が必要なレベル。


In [63]:
res = qa_chain.invoke('無限パンチとはなんですか？')
print(res['result'])



> Entering new RetrievalQA chain...

> Finished chain.
 無限パンチとは、漫画「ワンパンマン」に登場する技の名前です。


In [64]:
res = qa_chain.invoke('日本で一番高い山の名前と標高は何ですか？')
print(res['result'])



> Entering new RetrievalQA chain...

> Finished chain.
 日本で一番高い山は富士山で、標高は3776mです。
